# First Layer

In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

import datetime
import time
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

# open url in browser
driver = webdriver.Chrome(executable_path='/Users/fufufu/Documents/Python/Web Analytics/Selenium/chromedriver')
driver.get('https://www.stories.com/en_usd/clothing/all.html')
driver.find_element_by_xpath("/html/body/div[2]/div[10]/div/button").click() # close regis ad

# range may change

In [8]:
count = 0 

# while True:
for i in range(1,36): # click load more 20 times
    # Scroll, need to scroll 4 times in order to see load more button
    driver.execute_script("window.scrollBy(0,2000)")
    time.sleep(2)
    driver.execute_script("window.scrollBy(0,2000)") 
    time.sleep(2)
    driver.execute_script("window.scrollBy(0,2000)") 
    time.sleep(2)
    driver.execute_script("window.scrollBy(0,2000)") 
    time.sleep(2)
    # when reach bottom, click load more
    driver.find_element_by_xpath("/html/body/div[4]/div/div[4]/div[1]/button/span").click() # click load more
    count += 1

In [10]:
product_name = []
original_price = []
sale_price = []
product_url = []
brand = []
product_type = []
timestamp = []

soup = BeautifulSoup(driver.page_source,'html.parser')

for i in soup.find_all('div',{'o-product producttile-wrapper'}): 
    if i.find('p', {'class':'a-paragraph'}) != None: # product name
        product_name.append(i.find('p', {'class':'a-paragraph'}).text.strip())
    else:
        product_name.append(None)
    if i.find('span',{'class':'originalPrice'}).text != None: # original price
        original_price.append(i.find('span',{'class':'originalPrice'}).text)
    else:
        original_price.append(None)
    if i.find('span',{'class':'price'}).text != None: # sale price
        sale_price.append(i.find('span',{'class':'price'}).text)
    else:
        sale_price.append(None)
    if i.find('a',{'a-link no-styling plp-link-track'}).get('href') != None: # product url
        product_url.append(i.find('a',{'a-link no-styling plp-link-track'}).get('href'))
    else:
        product_url.append(None)
    # image_ur_low = [] 
    brand.append('&Other Stories') # brand
    product_type.append("Women's Clothing") # product type
    timestamp.append(date_today) # time_stamp 

In [11]:
column_names = ["product_name", "original_price", "sale_price",'product_link']
df = pd.DataFrame(columns = column_names)

df['product_name'] = product_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['product_link'] = product_url
df

,product_name,original_price,sale_price,product_link
0,Floral Button Puff Sleeve Cardigan,89.0,89.0,https://www.stories.com/en_usd/clothing/knitwe...
1,Oversized Wool Blend Overshirt,149.0,149.0,https://www.stories.com/en_usd/clothing/jacket...
2,Buttoned Boxy Faux Fur Sherpa Jacket,179.0,179.0,https://www.stories.com/en_usd/clothing/jacket...
3,Relaxed Collared V-Neck Sweater,89.0,89.0,https://www.stories.com/en_usd/clothing/knitwe...
4,Quilted Belted Banana Sleeve Coat,179.0,179.0,https://www.stories.com/en_usd/clothing/jacket...
...,...,...,...,...
1455,Glitter Velour Mini Dress,69.0,21.0,https://www.stories.com/en_usd/clothing/dresse...
1456,Stretchy Ribbed Wrap Dress,69.0,21.0,https://www.stories.com/en_usd/clothing/dresse...
1457,Abstract Leopard Print Belt Dress,85.0,25.0,https://www.stories.com/en_usd/clothing/dresse...
1458,Belted Velvet Leggings,99.0,30.0,https://www.stories.com/en_usd/clothing/trouse...


#  Second Layer

In [85]:
product_description = []
material = []
picture_name = []
color = []
image_url = []
counter = 0

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options, executable_path='/Users/fufufu/Documents/Python/Web Analytics/Selenium/chromedriver')
for link in product_url:
    
    # product_description
    if(link) == product_url[0]: # only need to click for first url
        driver.get(link)
        driver.find_element_by_xpath("/html/body/div[2]/div[10]/div/button").click() # close regis ad
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source,'html.parser')

        prod_des = str(soup.find('div', {'id':'product-description'}))
        prod_des = prod_des.replace('<div class="free-text" id="product-description"><p>','')
        prod_des = prod_des.replace('<div class="free-text" id="product-description"><div><div>','')
        prod_des = prod_des.replace('<div class="free-text" id="product-description"><div>','')
        product_description.append(prod_des[:prod_des.find('.')])
    else:
        driver.get(link)
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source,'html.parser')

        prod_des = str(soup.find('div', {'id':'product-description'}))
        prod_des = prod_des.replace('<div class="free-text" id="product-description"><p>','')
        prod_des = prod_des.replace('<div class="free-text" id="product-description"><div><div>','')
        prod_des = prod_des.replace('<div class="free-text" id="product-description"><div>','')
        product_description.append(prod_des[:prod_des.find('.')])
    # material
    if soup.find('span', {'id':'articleCompositions'}) != None:
        material.append(soup.find('span', {'id':'articleCompositions'}).text.replace('\n', '')) 
    else:
        material.append(None)
    # picture name
    picture_name.append('&Other_Stories_women_' + str(counter) + '.jpg') 
    # color
    if soup.find('div', {'class':'picked-color'}) != None:
        color.append(soup.find('div', {'class':'picked-color'}).text)
    else:
        color.append(None)
    # image url
    if soup.find('picture', {'class':'a-picture'}) != None:
        image_url.append('https:' + (soup.find('picture', {'class':'a-picture'}).find('source', {'media':'(min-width:1951px)'}).get('srcset').replace('amp','')))
    else:
        image_url.append(None)
    counter += 1
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="a1b9d23bbe92aba5b8afde9b1d154536")>>

In [87]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = product_name
second_pass.product_description = product_description
second_pass.price = original_price
second_pass.sales_price = sale_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,&Other Stories,Women's Clothing,Floral Button Puff Sleeve Cardigan,Puff sleeve cardigan with a sweetheart necklin...,89.0,89.0,Lilac,"Shell: Viscose 32%, Polyamide 19%, Wool 17%, ...",https://www.stories.com/en_usd/clothing/knitwe...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_0.jpg
1,&Other Stories,Women's Clothing,Oversized Wool Blend Overshirt,Oversized wool blend overshirt finished with d...,149.0,149.0,Dark Grey,"Shell: Wool 60%, Viscose 20%, Acrylic 20%",https://www.stories.com/en_usd/clothing/jacket...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_1.jpg
2,&Other Stories,Women's Clothing,Buttoned Boxy Faux Fur Sherpa Jacket,Buttoned boxy jacket featuring faux fur sherpa...,179.0,179.0,Black,"Lining: Polyester 63%, Wool 22%, Acrylic 15% ...",https://www.stories.com/en_usd/clothing/jacket...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_2.jpg
3,&Other Stories,Women's Clothing,Relaxed Collared V-Neck Sweater,Relaxed wide v-neck knit sweater finished with...,89.0,89.0,White,"Shell: Cotton 55%, Polyamide 25%, Polyester 1...",https://www.stories.com/en_usd/clothing/knitwe...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_3.jpg
4,&Other Stories,Women's Clothing,Quilted Belted Banana Sleeve Coat,Quilted coat featuring banana sleeves and a to...,179.0,179.0,Black,Shell: Polyester 100% Lining: Polyester 100% ...,https://www.stories.com/en_usd/clothing/jacket...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_4.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
1455,&Other Stories,Women's Clothing,Glitter Velour Mini Dress,"Glitter velour mini dress with long sleeves, d...",69.0,21.0,Black,"Lining: Cotton 100% Shell: Polyester 90%, Spa...",https://www.stories.com/en_usd/clothing/dresse...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_1455.jpg
1456,&Other Stories,Women's Clothing,Stretchy Ribbed Wrap Dress,Wrap style ribbed dress with a waistbelt,69.0,21.0,Black,"Shell: Viscose 88%, Spandex 12%",https://www.stories.com/en_usd/clothing/dresse...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_1456.jpg
1457,&Other Stories,Women's Clothing,Abstract Leopard Print Belt Dress,V-neck mini length dress with a waistbelt in ...,85.0,25.0,Abstract Leopard Print,Shell: Viscose 100%,https://www.stories.com/en_usd/clothing/dresse...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_1457.jpg
1458,&Other Stories,Women's Clothing,Belted Velvet Leggings,Slim-fitting velvet leggings defined by the la...,99.0,30.0,Black,"Pocket: Cotton 100% Shell: Cotton 70%, Viscos...",https://www.stories.com/en_usd/clothing/trouse...,https://lp.stories.com/app005prod?set=key[reso...,2020-12-23,&Other_Stories_women_1458.jpg


In [142]:
# drop rows which have na
droplist = second_pass[(second_pass['picture_url']).apply(lambda x: str(x) == 'None')].index.tolist()
second_pass = second_pass.drop(droplist)

In [143]:
second_pass.to_csv("&other-stories-women.csv", index=False, encoding = 'utf-8-sig')

In [145]:
import os
import requests as req

new_pd = pd.read_csv('&other-stories-women.csv', encoding='utf-8-sig')

os.mkdir('&other-stories-women')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('&other-stories-women/'+ "&Other_Stories_women_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1